# TalkingData AdTracking 데이터의 탐색적 분석 (Exploratory data analysis)
### 0. 데이터 필드 정보 및 의견
  - `ip`: ip address of click
  - `app`: app id for marketing
  - `device`: device type id of user mobile phone (e.g., iphone 6 plus, iphone 7, huawei mate 7, etc.)
  - `os`: os version id of user mobile phone
  - `channel`: channel id of mobile ad publisher
  - `click_time`: timestamp of click (UTC)
  - `attributed_time`: if user download the app for after clicking an ad, this is the time of the app download
    * 데이터 필드 중 유일하게 attributed_time 은 학습 데이터에만 존재하는데 다른 필드 데이터들로 하여금 테스트 데이터 내에 이 필드값의 예측치를 채워넣을 수 있을지 모르니  **반드시 확인하는 작업을 진행할 계획**입니다. 
  - `is_attributed`: the target that is to be predicted, indicating the app was downloaded

### 1. Upvote 1위의 EDA 자료를 요약해보았습니다. 
  > 자료 링크 - https://www.kaggle.com/anokas/talkingdata-adtracking-eda
  
  - 예측 확률에 따라 평가되는 LogLoss 방식과는 달리 0, 1을 확실히 구분해내는 것이 중요한 AUROC 평가에 해당합니다.
    > 확률의 리스케일링 (상수 및 승수의 추가) 는 스코어에 영향이 없습니다.
  
  - 압축된 1.8Gb 의 학습 데이터는 압축을 해제하면 무려 7.5Gb
    ```bash
    # File sizes
    train.csv                     7537.65MB
    test.csv                      863.27MB
    sample_submission.csv         195.58MB
    train_sample.csv              4.08MB
    
    # Line count:
    184903891 ../input/train.csv
    18790470 ../input/test.csv
    100001 ../input/train_sample.csv
    
    ```
    
  - 저자는 백만개 라인만 입력으로 사용하였습니다.
  - 데이터값이 전부 인코딩되어 있어서 다양한 차트를 통해 얻을 수 있는 인사이트가 **제한**됩니다.
  - 그러므로, 저자는 각 필드의 인코딩값의 유니크수, 인코딩 값별 빈도수의 분포도, 빈도수별 ID수를 나타내는 히스토그램 세가지 차트를 확인하였습니다.
    * 약 4만 정도 index 근처 몇개 ip 들이 월등히 높은 6000 이상의 클릭수를 보이고 있습니다.
    * app 은 명확하진 않지만 index 180 근처에 조금은 집중된 모습입니다.
    * device 는 index 290 근처에 매우 높은 클릭수를 보인 1개 정도의 device 값이 있는 것이 확인됩니다.
    * os 도 index 160 근처에서 높은 클릭수를 보인 2개 정도의 os 값이 확인됩니다.
  - 위 차트에서 확인한 것처럼 매우 극히 일부분의 필드값에 대해서만 Fraudulent 판정이 된 것으로 예상할 수 있습니다.
    * 백만개 라인 중 겨우 0.17% 가 Fraudulent 라고 판정된 데이터로 **결론적으로 매우 불균형적으로 학습될 수 있는 데이터이므로 over 또는 under 샘플링이 반드시 필요할 것으로 보입니다.**
    * **또는, 적절한 뉴럴넷의 하이퍼 파라미터 조정을 통해 이를 극복하거나 내부적으로 적절히 샘플링에 대응하는 Boolean 하이퍼 파라미터들을 포함하는 XGBoost, LightGBM 을 적용해보는 것이 좋은 방향 같습니다.**